In [1]:
import duckdb

### Question 1

We need to find player with no of gold medals won by them only for players who won only gold medals.


In [2]:
query = """
CREATE TABLE events (
ID int,
event varchar(255),
YEAR INt,
GOLD varchar(255),
SILVER varchar(255),
BRONZE varchar(255)
);

delete from events;

INSERT INTO events VALUES (1,'100m',2016, 'Amthhew Mcgarray','donald','barbara');
INSERT INTO events VALUES (2,'200m',2016, 'Nichole','Alvaro Eaton','janet Smith');
INSERT INTO events VALUES (3,'500m',2016, 'Charles','Nichole','Susana');
INSERT INTO events VALUES (4,'100m',2016, 'Ronald','maria','paula');
INSERT INTO events VALUES (5,'200m',2016, 'Alfred','carol','Steven');
INSERT INTO events VALUES (6,'500m',2016, 'Nichole','Alfred','Brandon');
INSERT INTO events VALUES (7,'100m',2016, 'Charles','Dennis','Susana');
INSERT INTO events VALUES (8,'200m',2016, 'Thomas','Dawn','catherine');
INSERT INTO events VALUES (9,'500m',2016, 'Thomas','Dennis','paula');
INSERT INTO events VALUES (10,'100m',2016, 'Charles','Dennis','Susana');
INSERT INTO events VALUES (11,'200m',2016, 'jessica','Donald','Stefeney');
INSERT INTO events VALUES (12,'500m',2016,'Thomas','Steven','Catherine');"""
duckdb.execute(query)
query = """SELECT * FROM events"""
duckdb.sql(query).show()

┌───────┬─────────┬───────┬──────────────────┬──────────────┬─────────────┐
│  ID   │  event  │ YEAR  │       GOLD       │    SILVER    │   BRONZE    │
│ int32 │ varchar │ int32 │     varchar      │   varchar    │   varchar   │
├───────┼─────────┼───────┼──────────────────┼──────────────┼─────────────┤
│     1 │ 100m    │  2016 │ Amthhew Mcgarray │ donald       │ barbara     │
│     2 │ 200m    │  2016 │ Nichole          │ Alvaro Eaton │ janet Smith │
│     3 │ 500m    │  2016 │ Charles          │ Nichole      │ Susana      │
│     4 │ 100m    │  2016 │ Ronald           │ maria        │ paula       │
│     5 │ 200m    │  2016 │ Alfred           │ carol        │ Steven      │
│     6 │ 500m    │  2016 │ Nichole          │ Alfred       │ Brandon     │
│     7 │ 100m    │  2016 │ Charles          │ Dennis       │ Susana      │
│     8 │ 200m    │  2016 │ Thomas           │ Dawn         │ catherine   │
│     9 │ 500m    │  2016 │ Thomas           │ Dennis       │ paula       │
│    10 │ 10

#### Solution


In [3]:
query = """
SELECT GOLD,COUNT(*) FROM events 
WHERE GOLD NOT IN (SELECT BRONZE FROM events UNION ALL SELECT SILVER FROM events)
GROUP BY GOLD
"""

duckdb.sql(query).show()

┌──────────────────┬──────────────┐
│       GOLD       │ count_star() │
│     varchar      │    int64     │
├──────────────────┼──────────────┤
│ Amthhew Mcgarray │            1 │
│ Charles          │            3 │
│ Ronald           │            1 │
│ Thomas           │            3 │
│ jessica          │            1 │
└──────────────────┴──────────────┘



#### Alternative Solution


In [4]:
query = """
WITH only_gold AS (
    SELECT GOLD
    FROM events
    EXCEPT
    SELECT SILVER
    FROM events
    EXCEPT
    SELECT BRONZE
    FROM events
)
SELECT events.GOLD,
    COUNT(*)
FROM events
    INNER JOIN only_gold ON events.GOLD = only_gold.GOLD
GROUP BY events.GOLD
"""
duckdb.sql(query).show()

┌──────────────────┬──────────────┐
│       GOLD       │ count_star() │
│     varchar      │    int64     │
├──────────────────┼──────────────┤
│ Amthhew Mcgarray │            1 │
│ Charles          │            3 │
│ Ronald           │            1 │
│ Thomas           │            3 │
│ jessica          │            1 │
└──────────────────┴──────────────┘



### Question 2

We need to find difference between 2 dates excluding weekends and public holidays . Basically we need to find business days between 2 given dates using SQL


In [ ]:
query = """
create table tickets
(
ticket_id varchar(10),
create_date date,
resolved_date date
);
delete from tickets;
insert into tickets values
(1,'2022-08-01','2022-08-03')
,(2,'2022-08-01','2022-08-12')
,(3,'2022-08-01','2022-08-16');
create table holidays
(
holiday_date date
,reason varchar(100)
);
delete from holidays;
insert into holidays values
('2022-08-11','Rakhi'),('2022-08-15','Independence day');
"""
duckdb.sql(query)

In [7]:
query="SELECT * FROM tickets"""
duckdb.sql(query).show()

┌───────────┬─────────────┬───────────────┐
│ ticket_id │ create_date │ resolved_date │
│  varchar  │    date     │     date      │
├───────────┼─────────────┼───────────────┤
│ 1         │ 2022-08-01  │ 2022-08-03    │
│ 2         │ 2022-08-01  │ 2022-08-12    │
│ 3         │ 2022-08-01  │ 2022-08-16    │
└───────────┴─────────────┴───────────────┘



In [8]:
query="SELECT * FROM holidays"""
duckdb.sql(query).show()

┌──────────────┬──────────────────┐
│ holiday_date │      reason      │
│     date     │     varchar      │
├──────────────┼──────────────────┤
│ 2022-08-11   │ Rakhi            │
│ 2022-08-15   │ Independence day │
└──────────────┴──────────────────┘



#### Solution

Trickier than I initially assumed. The concept of `JOIN` on dates with an `IN` condition requires an amount of practice


In [20]:
query = """
SELECT (
        DATE_DIFF('day', t.create_date, t.resolved_date) - (
            DATE_DIFF('week', t.create_date, t.resolved_date) * 2
        ) - COUNT(h.holiday_date)
    ) AS business_days
FROM tickets AS t
    LEFT JOIN holidays AS h ON h.holiday_date BETWEEN t.create_date AND t.resolved_date
GROUP BY t.create_date,
    t.resolved_date
"""
duckdb.sql(query).show()


┌───────────────┐
│ business_days │
│     int64     │
├───────────────┤
│             2 │
│             8 │
│             9 │
└───────────────┘



### Question 3

We need to find number of employees inside the hospital.


In [2]:
query = """
create table hospital ( emp_id int
, action varchar(10)
, time datetime);

insert into hospital values ('1', 'in', '2019-12-22 09:00:00');
insert into hospital values ('1', 'out', '2019-12-22 09:15:00');
insert into hospital values ('2', 'in', '2019-12-22 09:00:00');
insert into hospital values ('2', 'out', '2019-12-22 09:15:00');
insert into hospital values ('2', 'in', '2019-12-22 09:30:00');
insert into hospital values ('3', 'out', '2019-12-22 09:00:00');
insert into hospital values ('3', 'in', '2019-12-22 09:15:00');
insert into hospital values ('3', 'out', '2019-12-22 09:30:00');
insert into hospital values ('3', 'in', '2019-12-22 09:45:00');
insert into hospital values ('4', 'in', '2019-12-22 09:45:00');
insert into hospital values ('5', 'out', '2019-12-22 09:40:00');
"""
duckdb.sql(query)

query = """SELECT * FROM hospital"""
duckdb.sql(query).show()

┌────────┬─────────┬─────────────────────┐
│ emp_id │ action  │        time         │
│ int32  │ varchar │      timestamp      │
├────────┼─────────┼─────────────────────┤
│      1 │ in      │ 2019-12-22 09:00:00 │
│      1 │ out     │ 2019-12-22 09:15:00 │
│      2 │ in      │ 2019-12-22 09:00:00 │
│      2 │ out     │ 2019-12-22 09:15:00 │
│      2 │ in      │ 2019-12-22 09:30:00 │
│      3 │ out     │ 2019-12-22 09:00:00 │
│      3 │ in      │ 2019-12-22 09:15:00 │
│      3 │ out     │ 2019-12-22 09:30:00 │
│      3 │ in      │ 2019-12-22 09:45:00 │
│      4 │ in      │ 2019-12-22 09:45:00 │
│      5 │ out     │ 2019-12-22 09:40:00 │
├────────┴─────────┴─────────────────────┤
│ 11 rows                      3 columns │
└────────────────────────────────────────┘



##### Solution

A combination of CTEs with window functions does the trick pretty well


In [14]:
query = """
WITH ranked_attendance AS (
    SELECT h.*,
        ROW_NUMBER() OVER (
            PARTITION BY emp_id
            ORDER BY time DESC
        ) AS rn
    FROM hospital AS h
)
SELECT *
FROM ranked_attendance
WHERE rn = 1
    AND action = 'in';
"""
duckdb.sql(query).show()

┌────────┬─────────┬─────────────────────┬───────┐
│ emp_id │ action  │        time         │  rn   │
│ int32  │ varchar │      timestamp      │ int64 │
├────────┼─────────┼─────────────────────┼───────┤
│      2 │ in      │ 2019-12-22 09:30:00 │     1 │
│      3 │ in      │ 2019-12-22 09:45:00 │     1 │
│      4 │ in      │ 2019-12-22 09:45:00 │     1 │
└────────┴─────────┴─────────────────────┴───────┘



### Question 4

Write query to return all employees whose salary is same in same department

In [9]:
query = """
CREATE TABLE emp_salary
(
    emp_id INTEGER  NOT NULL,
    name NVARCHAR(20)  NOT NULL,
    salary NVARCHAR(30),
    dept_id INTEGER
);


INSERT INTO emp_salary
(emp_id, name, salary, dept_id)
VALUES(101, 'sohan', '3000', '11'),
(102, 'rohan', '4000', '12'),
(103, 'mohan', '5000', '13'),
(104, 'cat', '3000', '11'),
(105, 'suresh', '4000', '12'),
(109, 'mahesh', '7000', '12'),
(108, 'kamal', '8000', '11');
"""
duckdb.sql(query)

query = """SELECT * FROM emp_salary"""
duckdb.sql(query).show()

┌────────┬─────────┬─────────┬─────────┐
│ emp_id │  name   │ salary  │ dept_id │
│ int32  │ varchar │ varchar │  int32  │
├────────┼─────────┼─────────┼─────────┤
│    101 │ sohan   │ 3000    │      11 │
│    102 │ rohan   │ 4000    │      12 │
│    103 │ mohan   │ 5000    │      13 │
│    104 │ cat     │ 3000    │      11 │
│    105 │ suresh  │ 4000    │      12 │
│    109 │ mahesh  │ 7000    │      12 │
│    108 │ kamal   │ 8000    │      11 │
└────────┴─────────┴─────────┴─────────┘



Rather simple INNER JOIN at the end of the day

In [25]:
query = """
SELECT e.emp_id,
    e.name,
    ef.emp_id,
    ef.name
FROM emp_salary AS e
    INNER JOIN emp_salary as ef ON e.salary = ef.salary
    AND e.dept_id = ef.dept_id
    AND e.emp_id > ef.emp_id
"""
duckdb.sql(query).show()

┌────────┬─────────┬────────┬─────────┐
│ emp_id │  name   │ emp_id │  name   │
│ int32  │ varchar │ int32  │ varchar │
├────────┼─────────┼────────┼─────────┤
│    104 │ cat     │    101 │ sohan   │
│    105 │ suresh  │    102 │ rohan   │
└────────┴─────────┴────────┴─────────┘



In [ ]:
query = """

"""
duckdb.sql(query)


query = "SELECT * FROM "
duckdb.sql(query).show()